In [326]:
import numpy as np
import pandas as pd
import re
import ast
import matplotlib.pyplot as plt
#!pip install wordcloud
import math

In [227]:
#path = input('Enter path to master dataset')
path = 'MasterDatasetTemp_Preprocessed_Anonymized_small.csv'

In [257]:
df = pd.read_csv (path) 

In [229]:
df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'USN', '10thPercentage', '10thMarks',
       '12thPercentage', 'Branch', 'CGPA', 'CourseName', 'CourseGrade',
       'CompanyName_New', 'CTC', 'TierLevel', 'CoCurricularActivities',
       'EmploymentType', 'EventsParticipated', 'GeneralSkills', 'Languages',
       'MinorAttended', 'NoofInternships', 'InternshipProjectDomain',
       'InternshipCompany', 'NoofProjects', 'ProjectDetailDomain',
       'ProgLanguages', 'Publication', 'ResearchDomain', 'ScholarshipsKey',
       'SoftwareTools', 'VolunteeringWork', 'WorkshopsOrg', 'WorkshopsDomain',
       'AwardsNLPkey', 'ExternalCertificatesKey',
       'ExternalCertificatesDomain'],
      dtype='object')

In [230]:
df_courses = df[['CompanyName_New','CourseName','CourseGrade']]


In [334]:
df_companyDet = df[['CompanyName_New','CTC','TierLevel']]
df_companyDet = df_companyDet[df_companyDet['CompanyName_New'].notna()]
df_companyDet = df_companyDet.fillna(0)
#df_companyDet['CTC_avg'] = df_companyDet.groupby(['CompanyName_New'])['CTC'].apply(lambda x : x.mean())
#df_companyDet.groupby(['CompanyName_New'])['TierLevel']
df_ctc = df_companyDet.groupby('CompanyName_New')['CTC'].apply(lambda x : x.astype(float)[x!=0].mean()).to_frame()
df_tier = df_companyDet.groupby('CompanyName_New')['TierLevel'].apply(lambda x :x.astype(float)[x!=0].mean()).to_frame()
df_ctc.fillna(0,inplace =True)
df_tier.fillna(0,inplace = True)
df_tier = df_tier.apply(np.floor)

In [337]:
df_company_det = pd.concat([df_ctc, df_tier], axis=1)
df_company_det

,CTC,TierLevel
CompanyName_New,,
accenture,5.700000,2.0
airasia technology centre,10.350000,1.0
akamai technologies,16.609000,1.0
amadeus software labs,9.535714,1.0
amagi,15.000000,1.0
amazon,7.000000,1.0
analog devices,20.500000,1.0
app dynamics,39.200000,1.0
cisco systems,16.790000,1.0


In [231]:
df_courses.CompanyName_New.isnull().value_counts()

False    180
True      13
Name: CompanyName_New, dtype: int64

In [232]:
df_courses = df_courses[df_courses['CompanyName_New'].notna()]

In [233]:
#---------------------------------------------regularizing subjects -----------------------------------------------------------
# done'unified_engineering','unified_engineering_(jedi)',
# done'engineering_mechanics','engineering_mechanics-dynamics',
# done 'introduction_to_data_science','introduction_to_data_sciences',
# done  __ to _
# done system -> systems
# done enginering -> engineering
# done dbms -> data_base_management_systems
# done -  to -
# done & to and
# done metal_casting_and_welding', 'metal_casting_and_welding_processes',
# done 'power_electronics', 'power_electronics_and_its_applications_to_converters',
# done 'web_technologies-i','web_technologies-ii', 'web_technologies_i', 'web_technologies_ii', 'web_technology-_i',
# 'wireless_communication','wireless_network_communications'
# done processors to processor 
def regularize_courses_grades(l,lg):
    lg = regularize_grades(lg)
    l_new = []
    lg_new = []
    for i in range(0,len(l)):
        l[i] = l[i].replace('-',' ')
        l[i] = re.sub(' +',' ',l[i])
        l[i] = l[i].replace('&','and').replace('systems','system').replace('sciences','science').replace('processors','processor').replace('technology','technologies').replace('enginering','engineering').replace('1','i').replace('2','ii')
        l[i]= l[i].strip().replace(' ','_')
        if(l[i].count('laboratory')>0):
            continue
        if(lg[i] == -1):
            continue
        if(l[i]=='unified_engineering_(jedi)'):
            l[i]= 'unified_engineering'
        if(l[i]=='wireless_communication'):
            l[i]= 'wireless_network_communications'
        if(l[i] == 'engineering_mechanics_dynamics'):
            l[i] = 'engineering_mechanics'
        if(l[i].count('dbms')>0):
            l[i] = l[i].replace('dbms','data_base_management_systems')
        if(l[i].count('metal_casting_and_welding')>0):
            l[i] = 'metal_casting_and_welding'
        if(l[i].count('power_electronics')>0):
            l[i] = 'power_electronics'
        if(l[i].count('mini_project')>0):
            l[i] = 'mini_project'
        if(l[i].count('vlsi')>0):
            l[i] = 'vlsi'
        if(l[i].count('project_work')>0):
            l[i] = 'project_work'
            
        # in case of repetitions, take highest grade   
        if(l[i] in l_new):
            if(int(lg[i])>int(lg_new[l_new.index(l[i])])):
                lg_new[l_new.index(l[i])] = str(lg[i])
            continue       
        l_new.append(l[i])
        lg_new.append(str(lg[i]))
   
    #res = [i +'@@'+ j for i, j in zip(l_unique, lg_new)] 
    lc = '##'.join(l_new)
    lg = '##'.join(lg_new)
    res = lc+'@@'+lg
    return res
        

In [234]:
#---------------------------------------------regularizing grades -----------------------------------------------------------

def regularize_grades(l):
    d = {'s':6,'a':5,'b':4, 'c':3, 'd':2, 'e':1, 'f':0, 'ap': -1}
    for i in range(len(l)):
        l[i] = l[i].replace('"','')
        l[i] = d[l[i]]
    return l
        

In [235]:
#df_courses['CourseGrade_New'] = df_courses.apply(lambda row : regularize_grades( list(ast.literal_eval(row['CourseGrade']))), axis = 1) 

In [236]:
df_courses['CourseDetails'] = df_courses.apply(lambda row : regularize_courses_grades( list(ast.literal_eval(row['CourseName'])), list(ast.literal_eval(row['CourseGrade']))), axis = 1) 

In [237]:
df_courses[['CourseName_New','CourseGrade_New']] = df_courses.CourseDetails.str.split("@@",expand=True) 


In [238]:
df_courses

,CompanyName_New,CourseName,CourseGrade,CourseDetails,CourseName_New,CourseGrade_New
0,walmart labs,"['introduction to computing using python ', 'i...","['s', 's', 's', 'a', 's', 's', 's', 's', 's', ...",introduction_to_computing_using_python##proble...,introduction_to_computing_using_python##proble...,6##6##6##6##6##6##6##6##6##5##6##6##4##5##6##6...
1,themathcompany,"['introduction to computing using python', 'in...","['c', 'a', 'b', 'a', 's', 'a', 'b', 'a', 'a', ...",introduction_to_computing_using_python##proble...,introduction_to_computing_using_python##proble...,3##4##6##5##4##5##5##4##5##5##4##6##6##5##6##5...
2,nxp semiconductors,"['introduction to computing using python', 'in...","['c', 'a', 'b', 'a', 's', 'a', 'b', 'a', 'a', ...",introduction_to_computing_using_python##proble...,introduction_to_computing_using_python##proble...,3##4##6##5##4##5##5##4##5##5##4##6##6##5##6##5...
3,intuit,"['introduction to computing using python', 'in...","['b', 's', 'a', 'a', 's', 's', 'a', 'a', 'a', ...",introduction_to_computing_using_python##proble...,introduction_to_computing_using_python##proble...,4##5##6##6##5##5##5##5##6##6##6##6##6##6##6##6...
4,citrix,"['introduction to computing using python', 'in...","['b', 's', 'a', 'a', 'a', 's', 'a', 'b', 'a', ...",introduction_to_computing_using_python##proble...,introduction_to_computing_using_python##proble...,4##5##5##6##5##4##5##5##5##5##5##5##6##5##6##5...
5,novarties healthcare,"['introduction to computing using python', 'in...","['b', 's', 'b', 'a', 'b', 'a', 'b', 'b', 'b', ...",introduction_to_computing_using_python##proble...,introduction_to_computing_using_python##proble...,4##4##4##5##4##4##4##6##5##5##5##6##6##6##5##6...
6,cisco systems,"['introduction to computing using python', 'in...","['s', 's', 's', 's', 's', 's', 'a', 's', 's', ...",introduction_to_computing_using_python##proble...,introduction_to_computing_using_python##proble...,6##6##6##6##5##6##6##6##6##6##5##6##6##6##6##6...
7,optum,"['introduction to computing using python', 'in...","['s', 's', 's', 's', 's', 's', 'a', 's', 's', ...",introduction_to_computing_using_python##proble...,introduction_to_computing_using_python##proble...,6##6##6##6##5##6##6##6##6##6##5##6##6##6##6##6...
8,cisco systems,"['introduction to computing using python', 'in...","['s', 's', 's', 's', 'a', 's', 'b', 'a', 'a', ...",introduction_to_computing_using_python##proble...,introduction_to_computing_using_python##proble...,6##6##5##6##4##5##5##6##6##5##5##6##6##6##6##6...
9,optum,"['introduction to computing using python', 'in...","['a', 's', 's', 's', 'a', 's', 'b', 'a', 'b', ...",introduction_to_computing_using_python##proble...,introduction_to_computing_using_python##proble...,5##6##5##6##4##5##4##5##5##5##4##5##5##6##6##5...


In [239]:
#getting distinct list of all courses, after cleaning
course = df_courses.CourseName_New.apply(lambda x: pd.value_counts(x.split("##"))).sum(axis = 0).to_frame()
course_all = course.index.values.tolist()

#print(len(sorted(course_all)))
#sorted(course_all)

In [240]:
#getting distinct list of all courses, after cleaning
grades = df_courses.CourseGrade_New.apply(lambda x: pd.value_counts(x.split("##"))).sum(axis = 0).to_frame()
grades_all = grades.index.values.tolist()
#print(len(sorted(course_all)))
#sorted(course_all)
grades_all

['6', '5', '4', '3', '2', '1', '0']

In [241]:
# removing courses that appear in more than 75% of the data, since they will mostly be core courses
course = df_courses.CourseName_New.apply(lambda x: pd.value_counts(x.split("##"))).sum(axis = 0).lt(0.75*df_courses.shape[0]).to_frame()
course_distinct = course[course[0] == True]
course_distinct = course_distinct.index.values.tolist()
#print(len(course_distinct), course_distinct)

In [242]:
# function to find occurence of specific courses and creating new column for each unqiue course. Fill with 1/0 based on if course was taken or not
def keyword_matching_course(keywords,df):
    df = df.join(pd.DataFrame(0, columns=keywords, index=df.index))
    df['CourseNameKey'] = ''
    #print(df.shape)
    for key in keywords:
        #print(key)
        for i in range(0, len(df)):
            if(key in df.iloc[i]['CourseName_New'].split('##')):
                df.iloc[i, df.columns.get_loc('CourseNameKey')] =df.iloc[i, df.columns.get_loc('CourseNameKey')]+ key+'##'
                df.iloc[i, df.columns.get_loc(key)] = 1
    return df

In [243]:
df_course_details = df_courses
df_courses = keyword_matching_course(course_distinct,df_courses)

In [244]:
df_courses

,CompanyName_New,CourseName,CourseGrade,CourseDetails,CourseName_New,CourseGrade_New,advanced_machine_learning,internship,big_data,research_credits,...,design_of_machine_elements_ii,principles_of_flight,basics_of_automobile_engineering,metal_forming_processes,material_science_and_metallurgy,hands_on_course_on_digital_system_simulation_and_synthesis,linear_system,embedded_programming_with_arduino,adaptive_system,CourseNameKey
0,walmart labs,"['introduction to computing using python ', 'i...","['s', 's', 's', 'a', 's', 's', 's', 's', 's', ...",introduction_to_computing_using_python##proble...,introduction_to_computing_using_python##proble...,6##6##6##6##6##6##6##6##6##5##6##6##4##5##6##6...,1,1,1,1,...,0,0,0,0,0,0,0,0,0,advanced_machine_learning##internship##big_dat...
1,themathcompany,"['introduction to computing using python', 'in...","['c', 'a', 'b', 'a', 's', 'a', 'b', 'a', 'a', ...",introduction_to_computing_using_python##proble...,introduction_to_computing_using_python##proble...,3##4##6##5##4##5##5##4##5##5##4##6##6##5##6##5...,0,1,0,0,...,0,0,0,0,0,0,0,0,0,internship##digital_image_processing##principl...
2,nxp semiconductors,"['introduction to computing using python', 'in...","['c', 'a', 'b', 'a', 's', 'a', 'b', 'a', 'a', ...",introduction_to_computing_using_python##proble...,introduction_to_computing_using_python##proble...,3##4##6##5##4##5##5##4##5##5##4##6##6##5##6##5...,0,1,0,0,...,0,0,0,0,0,0,0,0,0,internship##digital_image_processing##principl...
3,intuit,"['introduction to computing using python', 'in...","['b', 's', 'a', 'a', 's', 's', 'a', 'a', 'a', ...",introduction_to_computing_using_python##proble...,introduction_to_computing_using_python##proble...,4##5##6##6##5##5##5##5##6##6##6##6##6##6##6##6...,0,1,1,0,...,0,0,0,0,0,0,0,0,0,internship##big_data##digital_image_processing...
4,citrix,"['introduction to computing using python', 'in...","['b', 's', 'a', 'a', 'a', 's', 'a', 'b', 'a', ...",introduction_to_computing_using_python##proble...,introduction_to_computing_using_python##proble...,4##5##5##6##5##4##5##5##5##5##5##5##6##5##6##5...,0,1,1,1,...,0,0,0,0,0,0,0,0,0,internship##big_data##research_credits##introd...
5,novarties healthcare,"['introduction to computing using python', 'in...","['b', 's', 'b', 'a', 'b', 'a', 'b', 'b', 'b', ...",introduction_to_computing_using_python##proble...,introduction_to_computing_using_python##proble...,4##4##4##5##4##4##4##6##5##5##5##6##6##6##5##6...,0,1,1,0,...,0,0,0,0,0,0,0,0,0,internship##big_data##introduction_to_software...
6,cisco systems,"['introduction to computing using python', 'in...","['s', 's', 's', 's', 's', 's', 'a', 's', 's', ...",introduction_to_computing_using_python##proble...,introduction_to_computing_using_python##proble...,6##6##6##6##5##6##6##6##6##6##5##6##6##6##6##6...,0,1,0,1,...,0,0,0,0,0,0,0,0,0,internship##research_credits##digital_image_pr...
7,optum,"['introduction to computing using python', 'in...","['s', 's', 's', 's', 's', 's', 'a', 's', 's', ...",introduction_to_computing_using_python##proble...,introduction_to_computing_using_python##proble...,6##6##6##6##5##6##6##6##6##6##5##6##6##6##6##6...,0,1,0,1,...,0,0,0,0,0,0,0,0,0,internship##research_credits##digital_image_pr...
8,cisco systems,"['introduction to computing using python', 'in...","['s', 's', 's', 's', 'a', 's', 'b', 'a', 'a', ...",introduction_to_computing_using_python##proble...,introduction_to_computing_using_python##proble...,6##6##5##6##4##5##5##6##6##5##5##6##6##6##6##6...,0,1,0,1,...,0,0,0,0,0,0,0,0,0,internship##research_credits##digital_image_pr...
9,optum,"['introduction to computing using python', 'in...","['a', 's', 's', 's', 'a', 's', 'b', 'a', 'b', ...",introduction_to_computing_using_python##proble...,introduction_to_computing_using_python##proble...,5##6##5##6##4##5##4##5##5##5##4##5##5##6##6##5...,0,1,0,0,...,0,0,0,0,0,0,0,0,0,internship##digital_image_processing##principl...


In [245]:
# getting total number of entries per company, and total sum of each course for each company
dfCourseCount = df_courses.groupby(['CompanyName_New'])[course_distinct].apply(lambda x : x.astype(int).sum())
dfCompanyCount = df_courses.groupby(['CompanyName_New'])['CourseName'].count().to_frame().rename(columns = {'CourseName': 'Total'}, inplace = False)

In [246]:
result = pd.concat([dfCompanyCount, dfCourseCount], axis=1)

In [247]:
# getting a rating by calculating, total number of students placed at a specific company who took that course/ total number of students placed at that company
result = result[course_distinct].div(result.Total, axis=0)
#result = result.reset_index(level=['CompanyName_New'])

In [248]:
originalRatingsData = result.stack().reset_index().rename(columns={'level_0':'CompanyName','level_1':'CourseName', 0:'Rating'})

In [249]:
# function to find occurence of specific courses and creating new column for each unqiue course. Fill with grades
def course_rating_grade(keywords,df):
    df = df.join(pd.DataFrame(-1, columns=keywords, index=df.index))
    #print(sorted(list(df.columns)))
    for key in keywords:
        for i in range(0, len(df)):
            if(key in df.iloc[i]['CourseName_New'].split('##')):
                temp_name = df.iloc[i]['CourseName_New'].split('##')
                temp_grade = df.iloc[i]['CourseGrade_New'].split('##')
                df.iloc[i, df.columns.get_loc(key)] = int(temp_grade[temp_name.index(key)])
    return df

In [250]:
result_grade = course_rating_grade(course_all, df_course_details)

In [251]:
result_grade

,CompanyName_New,CourseName,CourseGrade,CourseDetails,CourseName_New,CourseGrade_New,advanced_machine_learning,internship,big_data,research_credits,...,mechanics_of_machines_and_mechanisms,design_of_machine_elements_ii,principles_of_flight,basics_of_automobile_engineering,metal_forming_processes,material_science_and_metallurgy,hands_on_course_on_digital_system_simulation_and_synthesis,linear_system,embedded_programming_with_arduino,adaptive_system
0,walmart labs,"['introduction to computing using python ', 'i...","['s', 's', 's', 'a', 's', 's', 's', 's', 's', ...",introduction_to_computing_using_python##proble...,introduction_to_computing_using_python##proble...,6##6##6##6##6##6##6##6##6##5##6##6##4##5##6##6...,6,5,5,6,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,themathcompany,"['introduction to computing using python', 'in...","['c', 'a', 'b', 'a', 's', 'a', 'b', 'a', 'a', ...",introduction_to_computing_using_python##proble...,introduction_to_computing_using_python##proble...,3##4##6##5##4##5##5##4##5##5##4##6##6##5##6##5...,-1,5,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,nxp semiconductors,"['introduction to computing using python', 'in...","['c', 'a', 'b', 'a', 's', 'a', 'b', 'a', 'a', ...",introduction_to_computing_using_python##proble...,introduction_to_computing_using_python##proble...,3##4##6##5##4##5##5##4##5##5##4##6##6##5##6##5...,-1,5,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,intuit,"['introduction to computing using python', 'in...","['b', 's', 'a', 'a', 's', 's', 'a', 'a', 'a', ...",introduction_to_computing_using_python##proble...,introduction_to_computing_using_python##proble...,4##5##6##6##5##5##5##5##6##6##6##6##6##6##6##6...,-1,5,6,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,citrix,"['introduction to computing using python', 'in...","['b', 's', 'a', 'a', 'a', 's', 'a', 'b', 'a', ...",introduction_to_computing_using_python##proble...,introduction_to_computing_using_python##proble...,4##5##5##6##5##4##5##5##5##5##5##5##6##5##6##5...,-1,6,6,5,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
5,novarties healthcare,"['introduction to computing using python', 'in...","['b', 's', 'b', 'a', 'b', 'a', 'b', 'b', 'b', ...",introduction_to_computing_using_python##proble...,introduction_to_computing_using_python##proble...,4##4##4##5##4##4##4##6##5##5##5##6##6##6##5##6...,-1,5,5,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
6,cisco systems,"['introduction to computing using python', 'in...","['s', 's', 's', 's', 's', 's', 'a', 's', 's', ...",introduction_to_computing_using_python##proble...,introduction_to_computing_using_python##proble...,6##6##6##6##5##6##6##6##6##6##5##6##6##6##6##6...,-1,5,-1,6,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
7,optum,"['introduction to computing using python', 'in...","['s', 's', 's', 's', 's', 's', 'a', 's', 's', ...",introduction_to_computing_using_python##proble...,introduction_to_computing_using_python##proble...,6##6##6##6##5##6##6##6##6##6##5##6##6##6##6##6...,-1,5,-1,6,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
8,cisco systems,"['introduction to computing using python', 'in...","['s', 's', 's', 's', 'a', 's', 'b', 'a', 'a', ...",introduction_to_computing_using_python##proble...,introduction_to_computing_using_python##proble...,6##6##5##6##4##5##5##6##6##5##5##6##6##6##6##6...,-1,5,-1,6,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
9,optum,"['introduction to computing using python', 'in...","['a', 's', 's', 's', 'a', 's', 'b', 'a', 'b', ...",introduction_to_computing_using_python##proble...,introduction_to_computing_using_python##proble...,5##6##5##6##4##5##4##5##5##5##4##5##5##6##6##5...,-1,5,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [255]:
# getting total number of entries per company, and total sum of each course for each company
dfCourseGradeAvg = result_grade.groupby(['CompanyName_New'])[course_all].apply(lambda x : x.astype(int)[x!=-1].mean())
#dfCompanyCount = df_courses.groupby(['CompanyName_New'])['CourseName'].count().to_frame().rename(columns = {'CourseName': 'Total'}, inplace = False)

In [256]:
dfCourseGradeAvg.fillna(0,inplace = True)

In [254]:
originalGradeRatingsData = dfCourseGradeAvg.stack().reset_index().rename(columns={'level_0':'CompanyName','level_1':'CourseName', 0:'GradeRating'})